In [1]:
# Importamos las librerías necesarias para convertir el pdf a texto
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO

# Definimos la función que convierte el pdf a texto
def convert_pdf_to_txt(path):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()

    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)

    text = retstr.getvalue()

    fp.close()
    device.close()
    retstr.close()
    return text

In [2]:
# Guardamos el texto en un nuevo archivo
with open('Museos_Madrid.txt', 'w', encoding='utf-8') as f:
    text = convert_pdf_to_txt('Museos_Madrid.pdf')
    f.write(text)

In [3]:
# Leemos el texto por líneas
with open('Museos_Madrid.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()

In [ ]:
import pandas as pd

def get_general_data(centros, lines):
    # Obtenemos datos de cada centro
    data1 = {'Ubicacion':[], 'Centro':[], 'Direccion':[], 'Telefono':[], 'email':[],'Informacion':[]}

    clean_lines = [line.replace('\n', '') for line in lines]

    for centro in centros:

        data1['Ubicacion'].append(centro[0])
        data1['Centro'].append(centro[1])

        for i, line in enumerate(clean_lines):
            if line.startswith(centro[1]) and clean_lines[i-1].startswith(centro[0][:10]):
                for j in range(i+2, len(clean_lines)):
                    if len(data1['Informacion']) == len(data1['Centro']):
                        break
                    elif clean_lines[j].startswith('Información General'):
                        texto = ''
                        for k in range(j+2, len(clean_lines)):
                            if clean_lines[k] == '':
                                data1['Informacion'].append(texto)
                                break
                            else:
                                texto += ' ' + clean_lines[k]
                    if j == i + 2:
                        # Distintas opciones en función del tipo de dirección
                        if clean_lines[j+1].startswith('Tel'):
                            data1['Direccion'].append(clean_lines[j])
                            data1['Telefono'].append(clean_lines[j+1])
                            data1['email'].append(clean_lines[j+2])

                        elif clean_lines[j+2].startswith('Tel'):
                            data1['Direccion'].append(clean_lines[j] + ' ' + clean_lines[j+1])
                            data1['Telefono'].append(clean_lines[j+2])
                            data1['email'].append(clean_lines[j+3])

                        else:
                            data1['Direccion'].append(clean_lines[j] + ' ' + clean_lines[j+2])
                            data1['Telefono'].append(clean_lines[j+3])
                            data1['email'].append(clean_lines[j+4])

    centros_df = pd.DataFrame(data1, index=range(len(data1['Centro'])))

    return centros_df

def get_access_data(centros, lines):
    # Obtenemos accesibilidad de cada centro
    data2 = {'Centro':[]}

    clean_lines = [line.replace('\n', '') for line in lines]

    excepciones = ['Espacios y Actividades en la Naturaleza Accesibles para Todas las Personas',
                'Centros de interpretación / Casas del Parque y Aulas de la Naturaleza',
                'Ubicación',
                'Características destacables',
                'Zona de Picnic',
                '(cid:2) Dispone de paneles informati-',
                'trastado: Sí',
                'rrelieve: No',
                '(cid:2) Iconos homologados: no hay',
                'Señalización',
                'Sendero Guiado / Autoguiado',
                'pavimento',
                'Arboreto',
                'Monumentos, museos y puntos de interés turístico Accesibles para Todos',
                'Comunidad de Madrid',
                'Madrid',
                'silla de ruedas',
                'SSaallóónn  ddee  aaccttooss',
                'Planta baja',
                'Accesible',
                'SSeeññaalliizzaacciióónn',
                'nario sin asistencia.',
                'librería.',
                'confirme el registro de la llamada: No',
                'CCoonnssiiggnnaa',
                'Accesible  (cid:2) Hay plazas reservadas para usuarios de silla de rue-',
                ]

    found_cols = [] # Columnas que se añaden por centro

    # Extracción de datos
    for centro in centros:

        data2['Centro'].append(centro[1])
        print(len(data2['Centro']))

        for i, line in enumerate(clean_lines):
            if line.startswith(centro[1]) and clean_lines[i-1].startswith(centro[0][:10]):
                for j in range(i+2, len(clean_lines)):
                    if clean_lines[j] == 'Accesibilidad Visual':
                        break
                    elif clean_lines[j] == 'Accesibilidad Física':
                        for k in range(j+1, len(clean_lines)-1):
                            # Fin de los datos del centro
                            if clean_lines[k] == 'Accesibilidad Visual':
                                print(found_cols)
                                # Se mira el número de columnas y se añade 'NP' en las que existen pero no se han encontrado
                                for key in data2.keys():
                                    if key not in found_cols and key != 'Centro':
                                        data2[key].append('NP')
                                found_cols = [] # Se reinicia la cuenta
                                break
                            # Columna encontrada
                            elif clean_lines[k] != '' and clean_lines[k+1] == '' and clean_lines[k-1] == '' and clean_lines[k].replace(' ', '').isalpha() and '' not in clean_lines[k] and clean_lines[k] not in excepciones:
                                found_cols.append(clean_lines[k])
                                texto = ''
                                # Concatenando líneas de la respectiva columna
                                for n in range(k+2, len(clean_lines)):
                                    #print(len(data2['Centro']))
                                    if clean_lines[n] == '':
                                        if clean_lines[k] in data2.keys():
                                            data2[clean_lines[k]].append(texto.replace('- ', ''))
                                            break
                                        elif len(data2['Centro']) == 1:
                                            data2[clean_lines[k]] = [texto.replace('- ', '')]
                                            break
                                        else:
                                            data2[clean_lines[k]] = ['NP' for v in range(len(data2['Centro'])-1)] + [texto.replace('- ', '')]
                                            break
                                    # Nos saltamos los cambios de página.
                                    elif clean_lines[n] == '' and len(clean_lines[n+1]) == 1 or len(clean_lines[n]) == 1 and len(clean_lines[n+1]) == 1:
                                        continue
                                    else:
                                        texto += clean_lines[n] + ' '


    access_df = pd.DataFrame(data2, index=range(len(data2['Centro'])))

    return access_df

In [48]:
data = {'id_evento':[0,1,2,3,4]}

In [49]:
data['tipo_evento'] = ['Museo', 'Museo', 'Museo', 'Museo', 'Museo']

In [50]:
data['nombre_evento'] = ['Museo Nacional Centro de Arte Reina Sofía', 'Museo Nacional del Prado','Palacio Real de Madrid','Palacio Real de Aranjuez',
'Real Monasterio de San Lorenzo del Escorial',]

In [51]:
data['descripcion'] = ['''Uno de los más importantes museos de arte de España para obras realizadas desde finales
del siglo XIX a la actualidad, principalmente de artistas españoles: Dalí, Miró, Juan Gris, Julio Gonzá-
lez, Tapies, Chillida. La estrella es Guernica (1937) de Pablo Picasso. El museo está situado en un
bello edificio diseñado en tiempos de Carlos III por Sabatini, como Hospital General de San Carlos,
restaurado en los años ochenta del siglo pasado.''',
                        '''El Museo Nacional del Prado posee en la actualidad más de siete mil pinturas y alberga colecciones
destacadas de escultura clásica y moderna, de artes decorativas y de estampas y dibujos. La calidad
de su colección de pintura es prácticamente inigualable ya que en ella se encuentran las colecciones
más completas de Tiziano, Velázquez, Ribera, Murillo, Rubens, El Bosco o El Greco que se pueden
ver en una única institución.''',
                        '''El Palacio Real de Madrid es la residencia oficial de Su Majestad el Rey de España, que lo emplea en
las ceremonias de Estado, aunque no habita en él. 
El origen del palacio se remonta al siglo IX en el que el reino musulmán de Toledo construyó una edi-
ficación defensiva que después usaron los reyes de Castilla, sobre la que en el Siglo XVI, se constru-
yó el Antiguo Alcázar. 
Destruido este por un incendio en la Nochebuena de 1734, fue reconstruido por Felipe V.''',
                        '''Residencia de primavera de los monarcas españoles desde tiempos de
Felipe II, fue remodelado y ampliado en el siglo XVIII por Felipe V, Fernando VI y Carlos III para conver-
tirlo en un bellísimo palacio a orillas del Tajo, completado con destacadas actuaciones urbanísticas en
el entorno del edificio y en la propia villa de Aranjuez. En el siglo XIX continuaron las intervenciones de
la Corona, que harían también de sus jardines uno de los espacios más ricos y evocadores de los
Reales Sitios.''',
                        '''El Monasterio de El Escorial, construido entre 1563 y 1584 según los proyectos de Juan Bautista de
Toledo y Juan de Herrera, fue fundado por Felipe II con la intención de construir un mausoleo para su
padre, Carlos V, y como monumento conmemorativo de la victoria de San Quintín.
Las salas que pueden visitar PMR son los “Salones de Honor o Sala de Tapices”, la
“Basílica”, la “Sala de los Secretos y Sala Trinidad”, el “Claustro Principal”, la
“Iglesia Vieja” y las “Salas Capitulares”.''']

In [52]:
data['direccion'] = ['C/ Santa Isabel, 52 28012 Madrid', 'Paseo del Prado s/n 28014 Madrid','C/ Bailén, s/n 28071 Madrid', 
'Plaza de Parejas 28300 Aranjuez (Madrid)','Paseo de Juan Borbón y Battenberg s/n 28200 San Lorenzo del Escorial (Madrid)' ]

In [53]:
data['long'] = ['-3.6945998175650665','-3.6921700175648517','-3.7143120022211806','-3.60885477339763','-4.147716073379687']

In [54]:
data['lat'] = ['40.40805116002868', '40.41378177936541','40.41795497936503','40.03643670816857','40.58913854846469']

In [55]:
data['contacto'] = ['www.museoreinasofia.es', 'www.museodelprado.es', 'www.patrimonionacional.es', 'www.patrimonionacional.es', 'www.patrimonionacional.es']

In [56]:
data['accesibilidad'] = ['''El museo tiene dos accesos, uno en la fachada principal del edificio Sabatini y otro en el edificio
Nouvel.
Para acceder sin salvar escalones por el edificio Sabatini (Calle de Santa Isabel 52) debe borde-
arse la plaza de Sánchez Bustillo, dejando el edificio a la izquierda.
El acceso al edificio Nouvel (Pza/ Emperador Carlos V s/n, junto a Ronda de Atocha) no tiene
desniveles y es accesible para visitantes en silla de ruedas mediante rampas.''',
                        '''Los accesos sin desniveles corresponden a la Puerta de los Jerónimos en el edificio de la
ampliación y a la Puerta de Velázquez en el edificio de Villanueva. Para aproximarse a ambas
puertas sin salvar escalones se debe acceder por el Paseo del Prado.
Las Personas de Movilidad Reducida (PMR) pueden solicitar 72 horas antes, un permiso para estacionar su coche en la plaza
exterior del Museo, frente a la puerta de Goya.''',
                        '''Las 2 entradas del Palacio son accesibles para usuarios de silla de ruedas.
La entrada de la C/ Bailén es sin desniveles, la puerta permanece abierta en horario de visita y
tiene un ancho de paso > 80 cm. Se cruza un paso de control y escáner de 75 cm de ancho.
Por esta entrada se accede directamente al patio del Palacio.''',
                        '''La entrada por la que el público general accede al Palacio tiene cuatro escalones, dos de 10 cm
y dos de 18 cm.
Los usuarios en silla de ruedas acceden por una entrada alternativa, situada en la fachada late-
ral del Palacio y que corresponde con la salida de todos los visitantes.
La entrada tiene una rampa de 1,50 m de ancho, 1,94 m de longitud y una inclinación pronun-
ciada del 17,6%. No tiene pasamanos pero si un zócalo de protección a cada lado de 77 cm de
altura.''',
                        '''La entrada al Monasterio para el público general es por la Fachada Norte, en este acceso se
adquieren las los billetes de entrada.
Las PMR inician la visita a los salones de la planta baja por otra puerta de acceso ya que entre
la entrada principal y la primera sala hay 3 peldaños de 10 cm.
La puerta de acceso alternativa se encuentra junto a la de la entrada principal, la abre una per-
sona del personal de atención, si se solicita a la hora de comprar el billete.''']

In [33]:
len('''El Monasterio de El Escorial, construido entre 1563 y 1584 según los proyectos de Juan Bautista de
Toledo y Juan de Herrera, fue fundado por Felipe II con la intención de construir un mausoleo para su
padre, Carlos V, y como monumento conmemorativo de la victoria de San Quintín.
Las salas que pueden visitar PMR son los “Salones de Honor o Sala de Tapices”, la
“Basílica”, la “Sala de los Secretos y Sala Trinidad”, el “Claustro Principal”, la
“Iglesia Vieja” y las “Salas Capitulares”.''')

487

In [57]:
data['precio'] = ['Gratuito', 'Gratuito', 'Gratuito', 'Gratuito', 'Gratuito']

In [58]:
data['fecha_inicio'] = ['22-07-2021', '22-07-2021', '22-07-2021', '22-07-2021', '22-07-2021']

In [59]:
data['fecha_fin'] = ['22-07-2050', '22-07-2050', '22-07-2050', '22-07-2050', '22-07-2050']

In [ ]:
data['imagen_url'] = ['https://upload.wikimedia.org/wikipedia/commons/d/d9/Madrid_-_Museo_Nacional_Centro_de_Arte_Reina_Sof%C3%ADa_%28MNCARS%29_03.JPG',
'https://upload.wikimedia.org/wikipedia/commons/6/68/Museo_del_Prado_2016_%2825185969599%29.jpg',
'https://upload.wikimedia.org/wikipedia/commons/4/40/Palacio_Real_de_Madrid_-_03.jpg',
'https://upload.wikimedia.org/wikipedia/commons/4/49/Palacio_Real_de_Aranjuez_%285%29.jpg',
'http://turismomadrid.es/images/Portada/2020/02/monasterio-san-lorenzo-virtual-art-2020.jpg']

In [47]:
for key, val in data.items():
    try:
        print(key, len(val))
    except:
        print(key, val)

id_evento 6
tipo_evento 5
nombre_evento 5
descripcion 5
direccion 5
long 5
lat 5
contacto 5
accesibilidad 5
precio 5
fecha_inicio 5
fecha_fin 5


In [60]:
import pandas as pd

df = pd.DataFrame(data)

In [61]:
df

,id_evento,tipo_evento,nombre_evento,descripcion,direccion,long,lat,contacto,accesibilidad,precio,fecha_inicio,fecha_fin
0,0,Museo,Museo Nacional Centro de Arte Reina Sofía,Uno de los más importantes museos de arte de E...,"C/ Santa Isabel, 52 28012 Madrid",-3.6945998175650665,40.40805116002868,www.museoreinasofia.es,"El museo tiene dos accesos, uno en la fachada ...",Gratuito,22-07-2021,22-07-2050
1,1,Museo,Museo Nacional del Prado,El Museo Nacional del Prado posee en la actual...,Paseo del Prado s/n 28014 Madrid,-3.6921700175648517,40.41378177936541,www.museodelprado.es,Los accesos sin desniveles corresponden a la P...,Gratuito,22-07-2021,22-07-2050
2,2,Museo,Palacio Real de Madrid,El Palacio Real de Madrid es la residencia ofi...,"C/ Bailén, s/n 28071 Madrid",-3.7143120022211806,40.41795497936503,www.patrimonionacional.es,Las 2 entradas del Palacio son accesibles para...,Gratuito,22-07-2021,22-07-2050
3,3,Museo,Palacio Real de Aranjuez,Residencia de primavera de los monarcas españo...,Plaza de Parejas 28300 Aranjuez (Madrid),-3.60885477339763,40.03643670816857,www.patrimonionacional.es,La entrada por la que el público general acced...,Gratuito,22-07-2021,22-07-2050
4,4,Museo,Real Monasterio de San Lorenzo del Escorial,"El Monasterio de El Escorial, construido entre...",Paseo de Juan Borbón y Battenberg s/n 28200 Sa...,-4.147716073379687,40.58913854846469,www.patrimonionacional.es,La entrada al Monasterio para el público gener...,Gratuito,22-07-2021,22-07-2050


In [62]:
df.to_csv('info_museos_madrid.csv')

In [ ]:
data2 = {'Centro':['Museo Nacional Centro de Arte Reina Sofía', 'Museo Nacional del Prado','Palacio Real de Madrid','Palacio Real de Aranjuez',
'Real Monasterio de San Lorenzo del Escorial',]}

In [ ]:
data2['Acceso y taquillas de venta de entradas'] = ['''El museo tiene dos accesos, uno en la fachada principal del edificio Sabatini y otro en el edificio
Nouvel.
Para acceder sin salvar escalones por el edificio Sabatini (Calle de Santa Isabel 52) debe borde-
arse la plaza de Sánchez Bustillo, dejando el edificio a la izquierda.
El acceso al edificio Nouvel (Pza/ Emperador Carlos V s/n, junto a Ronda de Atocha) no tiene
desniveles y es accesible para visitantes en silla de ruedas mediante rampas.''',]

In [ ]:
data2['Vestíbulo y recepción'] = ['''Junto a las dos puertas de acceso del Museo se sitúa la taquilla de venta de entradas, la del edi-
ficio Sabatini está después de cruzar la puerta y la del edificio de la ampliación es exterior. 
La taquilla del edificio Sabatini tiene una altura de 11,,1100  mm de altura, no adaptada para usuarios
de silla de ruedas. La del edificio de la ampliación tiene una altura de 8800  ccmm, que permite la
aproximación lateral de una persona en silla de ruedas.
Todos los mostradores de información situados en las distintas plantas del Museo tienen una
altura de 8800  ccmm.''',]

In [ ]:
data2['Salas de exposiciones'] = ['''Las salas están distribuidas en las 44 plantas del Edificio Sabatini y en las 33  primeras plantas del
edificio de la ampliación, que se comunican por medio de ascensores y escaleras fijas. 
Todos los desniveles existentes en las salas de exposición disponen de rampas, a excepción de
la Sala E1 (Espacio Uno), ubicado en la planta 11ªª  del edificio Sabatini, que requiere salvar esca-
lones.
La comunicación entre ambos edificios se realiza por la 11ªª, 22ªª  y 33ªª planta del edificio Sabatini, por
medio de 22 rampas de 22,,5577  mm de ancho, 33,,4455  mm de longitud; ésta dispone de pasamanos a ambos lados. 
Las salas son diáfanas y con recorrido accesible. Disponen de bancos para hacer una pausa.
Aseo adaptado
Hay 33  cabinas de aseo para usuarios de silla de ruedas, situadas en la planta primera junto a la
galería principal, en la planta cuarta junto a la sala 440066 y la última en el edificio Nouvel, en una
entreplanta situada entre los niveles 0 y 1. Están señalizadas con el Símbolo Internacional de
Accesibilidad (SSIIAA).
La puerta de la cabina del edificio Nouvel abre hacia fuera y mide 9922  ccmm de ancho.''']

In [ ]:
data2['Inodoro'] = ['''El inodoro tiene un espacio libre de acercamiento por el lado dere-
cho de 11,,2200  mm de ancho y por el frente de 11,,4455  mm. El asiento está a
una altura de 4444  ccmm y el mecanismo de descarga a 9955  ccmm.
Tiene  una  barra  de  apoyo  abatible  a  la  derecha  y  una  fija  a  la
izquierda a 7700  ccmm de altura y con una separación entre ambas de
7755  ccmm.''']

In [ ]:
data2['Lavabo y accesorios del aseo'] = ['''Debajo del lavabo hay un espacio libre de 6666  ccmm de alto y 5500  ccmm  de
fondo, que permite la aproximación frontal con la silla de ruedas. El grifo es monomando.
La altura del borde inferior del espejo es de 11,,1177  mm y el resto de los accesorios del aseo están a
9900  ccmm de altura.''',]